In [5]:
%load_ext autoreload
%autoreload 2

---

In [1]:
from src.features.data_cleaning import *

pipeline = CleaningPipeline(path='../data/raw/twitter_tweets_raw.pkl')
cleaned_df = pipeline.run()

cleaned_df.head(5)

2023-05-23 15:10:56,193 - INFO - initialize pipeline and load & transform list of tweets in raw dataframe...
2023-05-23 15:11:02,453 - WARNING - starting data cleaning...
2023-05-23 15:11:02,469 - INFO - formating date...
2023-05-23 15:11:13,643 - WARNING - duplicate entries were found!
2023-05-23 15:11:13,643 - INFO - clean _no_duplicates...
2023-05-23 15:11:13,923 - WARNING - determine language for each entry...
  1%|▊                                                                          | 8873/854238 [00:31<50:17, 280.20it/s]

KeyboardInterrupt



---